In [1]:
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm
import os
import json
import numpy as np
import time
import random
import json
from concurrent.futures import ThreadPoolExecutor, as_completed
from datetime import datetime, timedelta

In [2]:
def generator():
    while True:
        yield

In [16]:
from datetime import timedelta, date

def daterange(date1, date2):
    for n in range(int ((date2 - date1).days)+1):
        yield date1 + timedelta(n)

start_dt = date(2005, 1, 1)
end_dt = date(2023, 9, 9)
dates = []
for dt in daterange(start_dt, end_dt):
    dates.append(dt.strftime("%Y-%m-%d"))
    
dates = dates[::-1]
len(dates)

6826

In [25]:
def crawl(url):
    r = requests.get(url)
    soup = BeautifulSoup(r.content, "lxml")
    a = soup.find_all('h2', {'class': 'post-title entry-title'})
    a = [a_.find('a').get('href') for a_ in a if a_ and len(a_.find('a').get('href', ''))]
    return a

In [26]:
from concurrent.futures import ThreadPoolExecutor, as_completed

max_worker = 10

data = []
for i in tqdm(range(0, len(dates), max_worker)):
    b = dates[i: i + max_worker]
    urls = [f'https://www.azhafizah.com/search?updated-max={t}T17%3A01%3A00%2B08%3A00&max-results=10' for t in b]
    done = False
    with ThreadPoolExecutor(max_workers=max_worker) as executor:
        futures = {executor.submit(crawl, url): url for url in urls}
        
        for future in as_completed(futures):
            r = future.result()
            data.extend(r)
            if not len(r):
                done = True
    
    if done:
        break

 74%|██████████████████████████████████████████████████████████████████████████████▉                            | 504/683 [12:08<04:18,  1.44s/it]


In [27]:
len(set(data))

5400

In [28]:
with open('azhafizah.com-urls.json', 'w') as fopen:
    json.dump(list(set(data)), fopen)